In [ ]:
import warnings
warnings.filterwarnings("ignore")

# Load the dataset
data = pd.read_csv('./data/laptopData.csv')

# Rename the column 'Unnamed: 0' to 'ID'
data.rename(columns={'Unnamed: 0': 'ID'}, inplace=True)

# Define the adjusted exchange rate based on a specific example
price_in_pkr_example = 47895.5232  # Example price in PKR
target_price_usd = 1000  # Desired average price in USD
adjusted_exchange_rate = price_in_pkr_example / target_price_usd

# Remove rows with NaN values in the 'Price' column
data = data[~data['Price'].isna()]


# Convert the 'Price' column to USD and replace the original column
data['Price'] = data['Price'] / adjusted_exchange_rate

# Create a copy of the original DataFrame
copy_data = data.copy()

# Display rows number
print("Number of rows in the original DataFrame:", data.shape[0])

#display data types of each column
print("Data types:" )
print(data.dtypes)

# Display the first few rows of the modified DataFrame
data.head()